![NERLab](http://www.fee.unicamp.br/sites/default/files/docentes/leoelias/imagens/logo_v1%28WEB%29.png)
--

This notebook encompasses a brief description and a mathematical model of the mechanisms associated with the generation of muscle force and surface electromyogram (sEMG) during voluntary isometric contractions. The main structure present in this model is the motor unit (MU) pool and also includes time and frequency domain metrics for post-processing analysis.

- Version 0.1: Jan. 6, 2019;
- Version 0.2: July 22, 2019;
- Version 0.3: October 3, 2019;
- Version 0.4 | Last update: October 27, 2020.

Authors:  
[Ricardo Gonçalves Molinari](mailto:molinari@gmail.com) and [Leonardo Abdala Elias](mailto:leoelias@fee.unicamp.br)

# Import libraries and create model objects

In [1]:
from nerlab import  fug, emg, force, util
import ipywidgets as wi

util.config_plots()
mnpool = fug.Phemo()
muscle_emg = emg.Emg_mod(mnpool)
muscle_force = force.Muscle_force(mnpool)
sim = util.sim_results(mnpool, muscle_emg, muscle_force)

# The motor neuron pool model
## Motor unit recruitment-firing rate relationship
This notebook  section generates the relatioship between the common drive input (composed of peripheral, spinal and descending excitatory inputs) to the motor neuron pool and each motor neuron firing rate of the pool. Each motor unit (MU) is composed by one motor neuron and it's innervated muscle fibers. This phenomelogical approach to motor unit pools is based on the work of [Fuglevand et al. (1993)](https://doi.org/10.1152/jn.1993.70.6.2470 "Models of recruitment and rate coding organization in motor-unit pools").  
In this model we define 3 types of MUs, which are:
- Type I motor units are the first recruited (lower excitation threshold) and have slow-contracting muscle fibres with lower amplitudes.
- Type IIa motor units are recruited latter (higher excitation threshold) and have fast-contracting muscle fibre with high amplitudes.
- Type IIb motor units are recruited last (highest excitation threshold) and have very fast-contracting muscle fibres with the highest amplitudes.

### Recruitment Range
___Recruitment range___  defines the relation between each motor unit recruitment threshold excitation by the equation:
$$RTE_i  =exp \left ( \frac{ln(RR)i}{n} \right )$$
where:
- $RTE_i$ is the recruitment threshold excitation of motor unit $i$;
- $RR$ is the Recruitment range;
- $i$ is the motor unit index;
- $n$ is the total number of Motor units.  


### Excitatory drive-firing rate relationship
The firing rate of a motor neuron is based on the work of [Fuglevand et al. (1993)](https://doi.org/10.1152/jn.1993.70.6.2470 "Models of recruitment and rate coding organization in motor-unit pools") and the equations that rule this model are:
$$\begin{align}
     if \quad e(t) \ge RTE_i , \quad  fr_i(t) &= g_{i}(e(t)-RTE_i) + MFR \\
     else \quad fr_i(t) & = 0\\
    \end{align}$$

where:
- $e(t)$ is the excitatory drive (or common drive) as function of time;
- $fr_i(t)$ is the mean firing rate of MU $i$ at instant of time $t$; 
- $g_{i}$ is the excitatory driving-firing rate gain for MU $i$;
- $MFR$ is the minimum firing rate;
- $PFR_i$ is the peak firing rate of motor unit with index $i$, defined by: 

    $$ PFR_i   = PFR_1 - PFRD \frac{RTE_i}{RTE_n} $$
    
- $PFRD$ is the peak firing rate difference between first and last recruited MU of the pool;

- $g_{i}$ is the excitatory drive-firing rate gain for MN $i$ and is  defined by the following:

$$ \begin{align}
    g_{i} &= g_{var}(i) \frac{PFR_i - MFR}{e_{max} - RTE_i} \\
    e_{max} &= \frac{RTE_n}{\tilde{e}_{LR}} \\
\end{align}$$

where:
- $e_{max}$ is the maximum excitatory drive to the MN pool;
- $\tilde{e}_{LR}$ is the normalized (by $e_{max}$) excitatory drive that recruit the last MN.
- $g_{var}(i)$ returns $n$ ($n$ = number of MUs) evenly spaced numbers between 1 and $g_{var}(n)$.

If the checkbox __Same gain for all MNs?__ is ticked, $g_{var}(i) = g_{var}(n)$, resulting in the same $g_{i}$ for all MNs.

For more information about motor units check out [Kernell (2006)](https://doi.org/10.1093/acprof:oso/9780198526551.001.0001 "The Motoneurone and its Muscle Fibres").

In [2]:
ui1,ws1 = util.wi1()
out1 = wi.interactive_output(mnpool.view_organization, ws1)
display(wi.HBox([ui1,out1]))

## Motor neuron pool common drive
The MN pool common drive and the recruitment-firing rate are used to generate the motor unit spike trains to the muscle. These spikes  will promote the isometric force  and electric activity fullfiled by the muscle. Different intesities(% of the maximum) may be explored with Trapezoidal or Senoidal time-series. To simulate constant isometric force, choose Trapezoidal, defines **_Plateau on_** as zero and ___Plateau off___ as maximum time. Recruitment and de-recruitment may been seen also using trapezoidal shape time-series , but ___Onset___ must be less the ___Plateau on___ and ___Offset___ must be less then ___Plateau off___. Periodic contractions may be explored with the Senoidal shape time-series, defining the maximum ___intensity___ and ___frequency___.

In [3]:
ws2,ui2 = util.wi2()
out2 = wi.interactive_output(mnpool.view_excitatory,ws2)
display(wi.HBox([ui2,out2]))

## Motor unit spike trains
The motor unit activity (spike train) resembles a sthocastic point process with a gaussian distribution. In this way, the work of [Fuglevand et al. (1993)](https://doi.org/10.1152/jn.1993.70.6.2470 "Models of recruitment and rate coding organization in motor-unit pools") simulates the spike times by the equation:

$$ t_{i,j}= \mu + \sigma. Z + t_{i,j-1}$$

where:
- $ t_{i,j}$ is the $j$ spike time of the MN $i$;
- $ \mu $ is the mean inter spike interval (ISI) of MN $i$;
- $ \sigma $ is the ISI standard deviation, defined as a function of the coefficient of variation ($cv$) and $mu$, $\sigma = \mu . cv$;
- $ Z$ represents how the spike time deviates from the mean value, been randomly selected between $-3.9$ and $3.9$ from a gaussian distribution;  

### Synchronism
Synchronous activity of MUs (i.e., near simultaneous APs produced by distinct MNs) is an emerging phenomenon caused by the common synaptic input to the MN pool. In this model, synchronization is imposed by the algorithm proposed by [Yao et al. 2000](https://doi.org/10.1152/jn.2000.83.1.441 "Motor-Unit Synchronization Increases EMG Amplitude and Decreases Force Steadiness of Simulated Contractions"). The original algorithm was modified in step 2 to deal with non-stationary excitatory drives, limiting synchronization process between spike times that ocurried in a maximum 15ms interval ([Shin et al. 2018](https://doi.org/10.1088/1741-2552/aa925d "Relative contribution of different altered motor unit control to muscle weakness in stroke: a simulation study")).

The scheme to add synchrony occurs after the sthocastic point process  that generates MU activity. The slider __Synch. level [%]__ controls the intensitify of the synchronization process, defining the percentage  used in each one of the following step:
1. For each MU in the pool, a percentage of the spike times are randomly picked to serve as reference for spike time adjustment;
2. Then, for each spike time picked in the last step, a percentage of the other recruited MUs are randomly picked to have it closest spike time adjusted to the reference.
3. The last step of the algorithm add some variability to the adjusted spike time, controlled by the slider __Synch. $\sigma$ [ms]__.

In [4]:
wi3 = util.wi3()
_ = wi.interact_manual(mnpool.view_neural_command, CoV = wi3[0], 
                       synch_level = wi3[1], sigma = wi3[2])

interactive(children=(FloatSlider(value=30.0, description='ISI $cv$ [%]:', layout=Layout(width='300px'), max=5…

# Muscle surface electromiography generation model
The muscle surface electromiography (EMG) derives from electric activity of motor unit muscle fibres , which are sparselly distributed across muscles. These signals travels across muscle, fat and skin tissue till reach the bipolar electrode (_red star_ in the following graphic) at skin surface. In this work, we modelled four factors that operates the EMG signal generation by the following: 
- Muscle cross-section morphology;
- Motor unit distribution within muscle cross-sectional area;
- Motor unit action potentials;
- Volume conductor (spatial filtering).

## Muscle cross-section morphology
Four different types of muscle cross-section morphology can be modeled to better represent the desired muscle by the following geometries: circle , ring, pizza, and ellipse. Also, cross-sectional area (___CSA___), fat thickness (___fat layer [mm]___) and skin thickness (___skin layer [mm]___) are features to be adjusted. The ring, pizza, and ellipse geometries needs two more configurations:
- ___Proportion___ defines, for ___ring___ geometry, the ratio between internal and external muscle radius and, for ___ellipse___ geometry,  the ratio between semi-minor and semi-major axes;
- ___Theta [rad]___ defines the "opening" of the choosen geometry. In other words, the angle formed between the vertical line (which passes through the origin) and the line that defines the "side" frontier of the muscle CSA, which also passes through the origin.

In [5]:
ws4,ui4 = util.wi4()
out4 = wi.interactive_output(muscle_emg.view_morpho, ws4)
display(wi.HBox([ui4, out4]))

## Motor unit distribution within muscle cross-sectional area
Many human muscles present some level of motor unit regionalization within muscle cross-sectional area ([Johnson et al., 1973](https://doi.org/10.1016/0022-510X(73)90023-3 "Data on the distribution of fibre types in thirty-six human muscles")). To simulate it, we define two different distributions, one for the motor units that innervate type I muscle fibres and for type II muscle fibres. Each motor unit position is defined by polar coordanates, i.d., the distance from the origin is called radius and the angle formed with the vertical is called polar angle. All motor units will have their polar angle defined by a stochastic process with uniform distribution  with limits depending on the choosen morphology. The radius coordinate is also defined by an stochastic process, but with a gaussian distribution.Here we have control over this radius coordinate distribution with the variable ___Type I $\mu$___, which defines the distribution center for Type I motor units and is normalized by the muscle external radius. ___Type II $\mu$___ is the same of  ___Type I $\mu$___, but defining the distribution center for Type II (a and b) motor units. On the other hand, ___Type I $\sigma$___ defines the standard deviation of the gaussian distribution of the radius for Type I motor units, also normalized by muscle external radius. ___Type II $\sigma$___ is the same of ___Type I $\sigma$___, but for the Type II (a and b) motor units.

### Muscle fiber ratio
The motor unit territory is highly correlated with the number of muscle fibres innervated by a single motor neuron, which varies across the motor unit pool ([Enoka and Fuglevand, 2001](https://doi.org/10.1002/1097-4598(200101)24:1<4::AID-MUS13>3.0.CO;2-F "Motor unit physiology: Some unresolved issues")). Usually, the muscle comprises many motor units with relatively few muscle fibres and few motor units with large quantity of muscle fibres.  This motor unit innervation across the pool can be modelled by the form:
$$y_i = y_1.exp \left (ln(R)\frac{i}{n} \right )$$
where:
- $y_i$ is the innervation number of the motor unit $i$;
- $y_1$ is the innervation number of the smallest motor unit;
- $n$ is the total number of motor units;
- $R$ is the ___innervation number ratio___ between the largest ($y_n$) and smallest ($y_1$) motor unit.




In [6]:
ui5,ws5 = util.wi5()
out5 = wi.interactive_output(muscle_emg.view_distribution, ws5)
display(wi.VBox([ui5, out5]))

## Motor unit action potentials
Motor unit action potentials (MUAP) observed from bipolar electrodes at skin surface can be represented by ___1rst or 2nd order___ Hermite-Rodriguez (HR) functions ([Lo Conte et al., 1994](http://doi.org/10.1109/10.335863 "Hermite Expansions of Compact Support Waveforms: Applications to Myoelectric Signals")). These functions fit well the MUAP shape and has been used in previous studies. The ___1rst order___ HR function models biphasic MUAPs while the ___2nd order___ function models triphasic MUAPs. Each MU has an equal probability to have it MUAP shape modelled as 1rst or 2nd order HR function. The following cell radio button ___Hermite Rodrigues function___ has the only purpose to control which function is displayed in the figure, i.e., it do not change the model configuration.
The HR functions are defined by the following equations:
$$\begin{align}
HR_1(t) &= \frac{A_M.(t-t_{AP})}{\lambda_M} e^{  - \left (\frac{t-t_{AP}}{\lambda_M} \right )^2}u(t-t_{AP}) \\
HR_2(t) &= A_M \left [1 - 2 \left ( \frac{t-t_{AP}}{\lambda_M} \right )^2 \right ] e^{- \left (\frac{t-t_{AP}}{\lambda_M} \right )^2} u(t-t_{AP})
\end{align}$$
where:
- $HR_i$ is the $i$-th order Hermite-Rodriguez Function;
- $A_M$ is the amplitude factor;
- $\lambda_M$ is the duration factor;
- $t$ is simulation time;
- $t_{AP}$ is the motor unit discharge time;
- $u$ is the Heaviside step function;

For each motor unit is atributed a distinct amplitude and duration factor. To define each MUAP amplitude factor, an exponential interpolation between the ___first MUAP amplitude [mV]___ and ___last MUAP amplitude [mV]___ is used. The same method is adopted to define the duration factors of each MUAP.

In [7]:
ui6,ws6 = util.wi6()
out6 = wi.interactive_output(muscle_emg.view_muap, ws6)
display(ui6, out6)

Output()

## Volume conductor (spatial filtering)
The volume conductor(characterized by the muscle, fat and skin tissue) filtering effect is modelled as isotropic and proportional to distance by the following equations ([Cisi and Kohn, 2008](http://doi.org/10.1007/s10827-008-0092-8 "Simulation system of spinal cord motor nuclei and associated nerves and muscles, in a Web-based architecture")): 
$$\begin{align}
V &= V_0. e^{ \left ( \frac{-d}{\tau_{at}} \right )} \\
T &= T_0.(1+C.d)
\end{align} $$
where:
- $V$ is the Motor unit action potential (MUAP) amplitude at skin surface;
- $V_0$ is the MUAP amplitude at the center of the motor unit territory;
- $d$ is the distance between the center of the motor unit territory and the center of bipolar electrode;
- $\tau_{at}$ is the ___amplitude attenuation factor [$mm^{-1}$]___;
- $T$ is the MUAP duration at the electrode;
- $T_0$ is the MUAP duration at the center of the motor unit territory;
- $C$ is the ___widening factor [$mm^{-1}$]___.

In [8]:
ui7,ws7 = util.wi7()        
out7= wi.interactive_output(muscle_emg.view_attenuation,ws7)
display(ui7,out7)

Output()

## Surface EMG Generation
The surface electromiography (EMG) is modelled as the summed activity of all filtered MUAPs and each MUAP train has uniform and equal probabilities to be represented by first or second order Hermite-Rodriguez Functions. Some noise is always present in experimentally measures and can be added as  user wish by adjusting the ___Noise level (Standard deviation)[mV]___ variable. Also,  we applied a 4th order Butterworth bandpass filter to clear the signal from undesired frequencies, which cut frequencies can be adjust by ___Bandpass filter low cut [Hz]___ and ___Bandpass filter high cut [Hz]___.

In [9]:
wi8 = util.wi8()
_ = wi.interact_manual(muscle_emg.view_semg, add_noise = wi8[0], noise_level = wi8[1], 
                       add_filter = wi8[4], bplc = wi8[2], bphc = wi8[3])

interactive(children=(Checkbox(value=True, description='Add Noise', layout=Layout(width='400px'), style=Descri…

## Surface EMG analysis
To analyse the EMG signal in time domain, which can be non-stationary,  we used in this work a simple moving RMS technique. It is a type of finite impulse response filter and is used to smooth the short-term fluctuations and highlight longer-term trend. For other time domain methods: [Challis and Kitney, 1990](https://doi.org/10.1007/BF02442601 "Biomedical signal processing (in four parts) - Part 1 Time-domain methods"). Furthermore, to analyse the EMG signal in frequency domain, we used two methods: Welch's periodogram ([Challis and Kitney, 1991](https://doi.org/10.1007/BF02446704 "Biomedical signal processing (in four parts) - Part Part 3 The power spectrum and coherence function")) and Spectrogram. Welch's method is an approach for spectral density estimation, which has the property to reduces noise in exchange of frequency resolution. This method is applied on the entire analysis interval and adequate for stationary signals. The Spectrogram method, which uses Short-time Fourier transform to be generated, is used to determine sinusoidal frequency of the EMG signal as it changes over time and is suitable for non-stationary signals.  
Still in the following cell, one can visualize each motor unit action potential train contribution, after volume conductor filtering, for the EMG signal by choosen the ___motor unit index #___.

In [10]:
ui9,ws9 = util.wi9(muscle_emg)
out9 = wi.interactive_output(muscle_emg.analysis, ws9)
display(ui9, out9)

Output()

# Muscle force generation model

## Motor Unit Twitch

Motor unit twitch is the mechanic response of muscle fibres to action potentials arriving from motor neurons. Under isometric conditions, it can be modelled as the impulsive response of a critically damped second-order system  ([Milner-brown et al., 1973](https://doi.org/10.1113/jphysiol.1973.sp010087 "The contractile properties of human motor units during voluntary isometric contractions")). The general expression for a second-order critically damped impulse response is:

$$ F(t) = P \frac{t}{T}exp(-t/T)u(t) \; , \quad t \in \Re$$

where:
- $F(t)$ is the force generated by one twitch;
- $P$ is the maximum force of the twitch;
- $T$ is the time-to-peak for force reach a maximum;
- $u(t)$ is the Heaviside step function.

The peak force generated by each MU twitch of the pool is directly related to its size and recruitment threshold, being modelled by the following equation ([Fuglevand et al., 1993](https://doi.org/10.1152/jn.1993.70.6.2470 "Models of recruitment and rate coding organization in motor-unit pools")).:

$$ P_i = P_0.e^{\frac{ln(RP).i}{n}}$$

where:
- $P_i$ is the peak twitch tension of the MU $i$;
- $P_0$ is the peak twitch force of the first MU;
- $RP$ is the the range of twitch forces;
- $n$ is the number of MUs in the pool.

The twitch time-to-peak organization of the MU pool can be modelled by an exponential curve, as proposed by the original paper ([Fuglevand et al., 1993](https://doi.org/10.1152/jn.1993.70.6.2470 "Models of recruitment and rate coding organization in motor-unit pools")):

$$ T_i=T_L \left ( \frac{1}{P_i} \right )^{1/log_{RT}RP} $$

where:
- $T_i$ is the twitch time for tension reach a maximum for motor unit $i$;
- $T_L$ is the largest $T_i$;
- $RT$ is the desired range of $T_i$ observed in the MU pool.

Assuming that the twitch time-to-peak is not related to MU size, one can set the radio button `Twitch duration` to `Random uniform`. This will result in a randomly, uniformly distributed, time-to-peak for each MU between the interval defined by $T_L/RT$ and $T_L$  (for more information about this topic, check [Bigland-Ritche et al., 1998](http:doi.org/10.1177/107385849800400413 "Contractile properties of human motor units: Is man a cat?")).

In [11]:
ui10,ws10 = util.wi10()
out10 = wi.interactive_output(muscle_force.view_mu_twitch, ws10)
display(ui10,out10)

Output()

## Motor unit tetanic force

The MU behavior has a number of nonlinear properties, including a change in the gain of the stimulus rate-MU isometric force system ([Milner-brown et al., 1973](https://doi.org/10.1113/jphysiol.1973.sp010087 "The contractile properties of human motor units during voluntary isometric contractions")). This relationship between stimulus rate and MU isometric force has a sigmoidal shape, saturating at a given stimulus rate. Usually, the stimulus rate for MU force saturation is bigger then 50 imp/s for muscles that control digits of the hand ([Fuglevand et al, 1999](https://doi.org/10.1152/jn.1999.81.4.1718 "Force-Frequency and Fatigue Properties of Motor Units in Muscles That Control Digits of the Human Hand")).


To simulate the saturation mechanisms, i.e., the non-linear relationship between stimulus rate and MU force, we used the following function ([Watanabe et al., 2013](https://doi.org/10.1152/jn.00073.2013 "Influences of premotoneuronal command statistics on the scaling of motor output variability during isometric plantar flexion")):

$$ F_{sat} (t)=\frac{1-exp(-c.F(t))}{1+exp(-c.F(t))} $$

To find which value of $c$ makes $F_{sat}(t)$ saturates at a desired stimulus rate, we first simulate the MU being stimulated at this rate and with unitary twitch amplitude ($P=1$). Then, we use the secant method (root-finding algorithm with secant lines with better approximations in each step) to find which $c$ value zeroes the function:

$$max(F_{sat}(t)) - 0.999 = 0$$ 

The shape of the stimulus rate-force relationship depends on the contractile properties of the MU. MU twitches with larger time-to-peak intervals tend to fuse faster when compared with shorter intervals. It is also important to note that the stimulus rate for MU force saturation  is much higher than the rate associated with maximum voluntary contractions (approximately a mean rate of 35 imp/s for the FDI muscle, [Enoka and Fuglevand, 2001](https://doi.org/https://doi.org/10.1002/1097-4598(200101)24:1<4::AID-MUS13>3.0.CO;2-F "Motor unit physiology: Some unresolved issues:")). 



In [12]:
ui11,ws11 = util.wi11(muscle_force)
out11 = wi.interactive_output(muscle_force.view_saturation, ws11)
display(ui11, out11)

Output()

## Force Generation

At this point, all of the force parameters are defined and simulation is ready to start (`run interact` button). To speed up simulation process, we used the discrete-time version of the spike train and twitch models. The spike train was viewed as an uniform pulse train of Kronecker deltas:

$$ s(n) = \sum_j \delta(n-n_j) \; , \quad n \in Z$$

where:
- $n_j$ is the $j$ MU spike time;
- $\delta(n)$ is the Kronecker delta function.

The twitch model was seen as a linear time-invariant system and it's sampled version is obtained with the impulse invariance technique. Then, the z-transform  is used to obtain the difference equation that describes the linear part of the force developed by one MU ([Cisi and Kohn, 2008](https://doi.org/10.1007/s10827-008-0092-8 "Simulation system of spinal cord motor nuclei and associated nerves and muscles, in a Web-based architecture.")):

$$     \begin{split}
F_{MU_i}(n) =  2 \, exp \left( \frac{- \Delta t}{T_{i}} \right ) \, F_{MU_i} (n-1) \, -exp \left ( \frac{-2 \Delta t}{T_i} \right )F_{MU_i}(n-2)   \\
 + \frac{\displaystyle P_i \, \Delta t^2}{\displaystyle T_i} exp \left (  1- \frac{\Delta t}{T_i} \right ) s(n-1)
    \end{split} $$

where:
- $F_{MU_i}(n)$ is the force of MU $i$ at the instant $n$;
- $\Delta t$ is the integration time step;

The last step in MU force generation is to add the saturation mechanism, described by the sigmoidal function $F_{sat}(t)$. Finally, muscle force is computed as the sum of all MU forces.

In [13]:
wif = wi.interact_manual(muscle_force.gen_force)

interactive(children=(Button(description='Run Interact', style=ButtonStyle()), Output()), _dom_classes=('widge…

## Force Analysis

Similar to the EMG analysis, the following cell uses Welch's periodogram and Spectrogram to analyse the muscle force signal in spectral domain. In time domain, standard deviation is used to analyse muscle force. It is also possible to observe single MU force contribution by ticking `Add motor unit contribution` in the `Motor unit Force` tab.

In [14]:
ui12, ws12 = util.wi12(muscle_force)
out12 = wi.interactive_output(muscle_force.analysis, ws12)
display(ui12, out12)

Output()

# Save Simulation Results
The configuration used to simulate this neuromuscular system and it's results can be saved at the following folder: `\simulation_results\folder_name`.The  _folder name_ can be choose in the following cell. To save it, just click on ___Run interact___ button.


In [15]:
ws13 = util.wi13()
_ = wi.interact_manual(sim.save_results, save_conf = ws13[0], save_emg = ws13[2], 
                          save_force = ws13[3], folder_name = ws13[4], save_spikes = ws13[1])

interactive(children=(Checkbox(value=True, description='Save Simulation Config.'), Checkbox(value=True, descri…